In [298]:
#Import required modules
from flask import Flask, render_template,send_file,send_from_directory,request, jsonify
from logging import FileHandler,WARNING
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from io import BytesIO
import base64

In [308]:
import session_info

In [309]:
session_info.show()

In [305]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [299]:
pd.options.display.max_columns=None
pd.options.display.max_colwidth=None

### Data Sources

In [300]:
# Import data sources
# RedCross Appeals
df_appeal = pd.read_csv('appeal.csv')
# RedCross Appeal Docs
df_appeal_docs = pd.read_csv('appealdocs.csv')

# Operational data extracted using NLP pipeline
df_flood = pd.read_csv('Flood_appeals_flood_budget_v5.csv')



### Data Preprocessing

#### Processing for operational data

In [301]:
# Drop Duplicates from Flood data source
df_flood.dropna(subset=['code'],inplace=True)
df_flood = df_flood[['code','type','document_url', 'Budget_TOTAL EXPENDITURE',
       'Sub_Budget_Total Consultants & Professional Fees',
       'Sub_Budget_Total General Expenditure',
       'Sub_Budget_Total Indirect Costs',
       'Sub_Budget_Total Land, vehicles & equipment',
       'Sub_Budget_Total Logistics, Transport & Storage',
       'Sub_Budget_Total Operational Provisions',
       'Sub_Budget_Total Programme Support', 'Sub_Budget_Total Personnel',
       'Sub_Budget_Total Relief items, Construction, Supplies',
       'Sub_Budget_Total Services', 'Sub_Budget_Total Supplies',
       'Sub_Budget_Total Transport & Storage',
       'Sub_Budget_Total Workshops & Training', 'TOTAL EXPENDITURE',
       'Sub_Exp_Total General Expenditure',
       'Sub_Exp_Total Land, vehicles & equipment',
       'Sub_Exp_Total Logistics, Transport & Storage',
       'Sub_Exp_Total Operational Provisions',
       'Sub_Exp__Total Programme Support', 'Sub_Exp__Total Services',
       'Sub_Exp_Total Supplies', 'Sub_Exp_Total Transport & Storage',
       'Sub_Exp_Total Workshops & Training', 'Sub_Exp_Total Indirect Costs',
       'Sub_Exp_Total Personnel',
       'Sub_Exp_Total Relief items, Construction, Supplies',
       'Sub_Exp_Consultants Professional Fees', 'Filename_y',
       'Var_Grand Total', 'Var_Grand Total Distributed', 'Budget_Grand Total',
       'ExpT1_Grand Total', 'Expenditure']]
df_flood.drop_duplicates(inplace=True)
tmp1 = df_flood#.loc[df_flood.duplicated(subset=['code'],keep=False)].sort_values('code')#.to_csv('tt.csv')
tmp1['document_url'] = tmp1['document_url'].str.lower()
tmp1['type'] = tmp1['type'].str.lower()

#resultant dataframe
df_output = pd.DataFrame()

#Drop based on Final Report
tmp = tmp1.loc[(tmp1['type']=='final report')]
df_output = df_output.append(tmp,ignore_index=True)
codes_processed = list(df_output['code'].unique())
codes_all = list(tmp1['code'].unique())
codes_remaining = list(set(codes_all)-set(codes_processed))


#Drop based on file name
tmp = tmp1.loc[tmp1['code'].isin(codes_remaining)]
tmp = tmp.loc[(tmp['document_url'].str.endswith('fr.pdf')) & (tmp['document_url'].str[-7].str.isdigit())]
df_output = df_output.append(tmp,ignore_index=True)
codes_p = list(df_output['code'])
codes = list(tmp1['code'].unique())
codes = list(set(codes)-set(codes_p))
tmp = tmp1.loc[tmp1['code'].isin(codes)]         
#Drop based on total expenditure
tmp = tmp.sort_values(by=['code','TOTAL EXPENDITURE'],ascending=False)
tmp.drop_duplicates(subset='code',keep='first',inplace=True)
df_output = df_output.append(tmp,ignore_index=True)

# Finall flood dataframe copy without duplicates.
df_flood = df_output.copy()
df_flood.shape

C:\Users\User\AppData\Local\Temp\ipykernel_1932\3478276762.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_output = df_output.append(tmp,ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_1932\3478276762.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_output = df_output.append(tmp,ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_1932\3478276762.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_output = df_output.append(tmp,ignore_index=True)


(510, 37)

#### Appeals Docs processing

In [302]:
#Filter appeal docs where final report is present
df_appeal_docs['name'] = df_appeal_docs['name'].str.lower()
df_appeal_docs_final = df_appeal_docs.loc[df_appeal_docs['name'].str.contains('final')]

df_appeal_docs_final = df_appeal_docs_final.sort_values(by=['appeal','created_at'],ascending=False)
df_appeal_docs_final.drop_duplicates(subset='appeal',keep='first',inplace=True)


#### Merging dataframes

In [293]:
# Merging appeal and appeal docs
df_merge = pd.merge(df_appeal,df_appeal_docs_final,how='left',left_on='code',right_on='appeal')
# Merging Flood operational dataframe
df_merge = pd.merge(df_merge,df_flood,how='left',left_on='code',right_on='code')
print(df_merge.shape)

(3735, 87)


#### Preparing dataframe for finding similar disasters from past

In [294]:
# Feature Engineering. create ratio of amount funded and number of beneficiaries.
df_merge['funded_ben'] = df_merge['amount_funded']/df_merge['num_beneficiaries']

# Filter nonzero rows for amount funded, amount requested and number of beneficiaries.
df_merge_nonzero = df_merge.loc[(df_merge['amount_funded']>0) & (df_merge['num_beneficiaries']>0) & (df_merge['amount_requested']>0)]
df_merge_nonzero['start_date'] = pd.to_datetime(df_merge_nonzero['start_date'])
df_merge_nonzero['start_year'] = df_merge_nonzero['start_date'].dt.year

C:\Users\User\AppData\Local\Temp\ipykernel_1932\3310098619.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merge_nonzero['start_date'] = pd.to_datetime(df_merge_nonzero['start_date'])
C:\Users\User\AppData\Local\Temp\ipykernel_1932\3310098619.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merge_nonzero['start_year'] = df_merge_nonzero['start_date'].dt.year


### Common Functions

In [295]:
# Function to calculate zscore differently for country and region
# Input <TBD>
def get_zscore(row,param):
    for groupby in ['country.name','region.region_name']:
        all_values=df_merge_nonzero.loc[(df_merge_nonzero[groupby]==row[groupby]) & (df_merge_nonzero['dtype.name']==row['dtype.name'])][param].values.tolist()
        zscore = stats.zscore(all_values)
        df_merge_nonzero.loc[(df_merge_nonzero[groupby]==row[groupby]) & (df_merge_nonzero['dtype.name']==row['dtype.name']),f'f_b_{groupby}_zscore']=zscore
        df_merge_nonzero.loc[(df_merge_nonzero[groupby]==row[groupby]) & (df_merge_nonzero['dtype.name']==row['dtype.name']),f'f_b_{groupby}_count']=len(all_values)
        df_merge_nonzero.loc[(df_merge_nonzero[groupby]==row[groupby]) & (df_merge_nonzero['dtype.name']==row['dtype.name']),f'f_b_{groupby}_mean']=np.mean(all_values)
        df_merge_nonzero.loc[(df_merge_nonzero[groupby]==row[groupby]) & (df_merge_nonzero['dtype.name']==row['dtype.name']),f'f_b_{groupby}_std']=np.std(all_values)
        
# Function to find closest rows for a given value     
def find_closest(number,values):
    diffs = [(value, abs(number - value), index) for index, value in values.iteritems()]
    # Sort the list by the difference
    diffs.sort(key=lambda x: x[1])
    # Return the first 5 values and indices from the sorted list
    return [(x[0], x[2]) for x in diffs[:5]]

#Function to get similar data
# Input parameters : Amount Funded, Number of beneficiaries, Disaster Type, Country, 
#                   Region, DataFrame from which similar data is required, Columns to display
# Output: 
#    1.dataframe 1 with similar data from past based on country
#    2.dataframe 2 with similar data from past based on region
#    3. Image 1 - based on country, visualizes the current input relative to all disasters in past yearwise.
#    4. Image 2 - based on region, visualizes the current input relative to all disasters in past yearwise.
def get_simlilar_data(amount_funded,num_beneficiaries,dtype,country,region,df_source,cols):
    f_b_ratio = amount_funded/num_beneficiaries
    print(f"Funded_Beneficiary_Ratio: {f_b_ratio}, Amount Funded: {amount_funded}, Num_Beneficiaries: {num_beneficiaries}, DisasterType: {dtype}, Country: {country}, Region: {region}")
    filter_col = ''
    df_country=pd.DataFrame()
    df_region=pd.DataFrame()
    img_country = BytesIO()
    img_region = BytesIO()
    for groupby in ['country.name','region.region_name']:
        if groupby == 'country.name':
            filter_col = country
        else:
            filter_col = region
        print(f"Closest by {groupby}")
        tmp_all = df_source.loc[(df_source[groupby]==filter_col) & (df_source['dtype.name']==dtype)][['country.name','start_date','funded_ben',f'f_b_{groupby}_mean',f'f_b_{groupby}_std',f'f_b_{groupby}_zscore']]
        tmp = tmp_all.head(1)
        historic_mean = tmp[f'f_b_{groupby}_mean'].item()
        historic_std = tmp[f'f_b_{groupby}_std'].item()
        zscore = (f_b_ratio - historic_mean)/historic_std
        values = tmp_all[f'f_b_{groupby}_zscore']
        closest_rows = find_closest(zscore,values)
        indexes = [index[1] for index in closest_rows]
#         display(df_merge_nonzero.loc[indexes][cols])
        if groupby == 'country.name':
            df_country = df_source.loc[indexes][cols]
            
            plt.figure(figsize=(20,5))
            plt.subplot(1, 2, 1)
            sns.histplot(
              tmp_all, x=tmp_all['start_date'].dt.year, y="funded_ben",
              bins=30, discrete=(True, False), log_scale=(False, True),
              cbar=True, cbar_kws=dict(shrink=.75)
              )
            plt.axhline(y = f_b_ratio, color ="red", linestyle ="--")
            plt.subplot(1, 2, 2)
            sns.histplot(
              tmp_all, x=tmp_all['start_date'].dt.year, y="funded_ben",
              bins=30, discrete=(True, False), log_scale=(False, False),
              cbar=True, cbar_kws=dict(shrink=.75)
              )
            plt.axhline(y = f_b_ratio, color ="red", linestyle ="--")

            img_country = BytesIO()
            plt.savefig(img_country, format='png')
            img_country.seek(0)
            
            plt.close()
            
        else:
            df_region = df_source.loc[indexes][cols]
            plt.figure(figsize=(20,5))
            plt.subplot(1, 2, 1)
            sns.histplot(
              tmp_all, x=tmp_all['start_date'].dt.year, y="funded_ben",
              bins=30, discrete=(True, False), log_scale=(False, True),
              cbar=True, cbar_kws=dict(shrink=.75)
              )
            plt.axhline(y = f_b_ratio, color ="red", linestyle ="--")
            plt.subplot(1, 2, 2)
            sns.histplot(
              tmp_all, x=tmp_all['start_date'].dt.year, y="funded_ben",
              bins=30, discrete=(True, False), log_scale=(False, False),
              cbar=True, cbar_kws=dict(shrink=.75)
              )
            plt.axhline(y = f_b_ratio, color ="red", linestyle ="--")

            img_region = BytesIO()
            plt.savefig(img_region, format='png')
            img_region.seek(0)
            
            plt.close()
        
        
        
    img_data_country = base64.b64encode(img_country.read()).decode() 
    img_data_region = base64.b64encode(img_region.read()).decode()
    df_country.rename(columns={'funded_ben':'funded/ben'},inplace=True)
    df_region.rename(columns={'funded_ben':'funded/ben'},inplace=True)
    return df_country,df_region,img_data_country,img_data_region

#### Testing

In [296]:
# Calculate zscore for ratio of amount funded and number of beneficiaries.
cols=['funded_ben','amount_funded','amount_requested','num_beneficiaries','atype_display','country.name','country.society_name','dtype.name','region.region_name','start_year']
df_merge_nonzero.apply(lambda row:get_zscore(row,'funded_ben'),axis=1)
get_simlilar_data(539063.00,40000,'Flood','India','Asia Pacific',df_merge_nonzero,cols)
# get_simlilar_data(539063.00,40000,'Epidemic','Cameroon','Africa',df_merge_nonzero,cols)

Funded_Beneficiary_Ratio: 13.476575, Amount Funded: 539063.0, Num_Beneficiaries: 40000, DisasterType: Flood, Country: India, Region: Asia Pacific
Closest by country.name
Closest by region.region_name


(      funded/ben  amount_funded  amount_requested  num_beneficiaries  \
 3      13.476575       539063.0          539063.0              40000   
 1029   13.205100       396153.0          396153.0              30000   
 748    12.933320       323333.0          323333.0              25000   
 1489   14.332520       358313.0          358313.0              25000   
 191    12.430940       621547.0          621547.0              50000   
 
      atype_display country.name      country.society_name dtype.name  \
 3             DREF        India  Indian Red Cross Society      Flood   
 1029          DREF        India  Indian Red Cross Society      Flood   
 748           DREF        India  Indian Red Cross Society      Flood   
 1489          DREF        India  Indian Red Cross Society      Flood   
 191           DREF        India  Indian Red Cross Society      Flood   
 
      region.region_name  start_year  
 3          Asia Pacific        2023  
 1029       Asia Pacific        2014  
 74

## End

## Flask App

In [297]:


app = Flask(__name__)

disaster_types=['']+list(df_merge_nonzero['dtype.name'].unique())
regions = list(df_merge_nonzero['region.region_name'].unique())
grouped_regions = df_merge_nonzero.groupby('region.region_name')['country.name'].apply(list)
grouped_regions = grouped_regions.to_dict()
cols=['funded_ben','amount_funded','amount_requested','num_beneficiaries','atype_display','country.name',
      'country.society_name','dtype.name','region.region_name','start_year']
cols_flood=['Sub_Budget_Total Consultants & Professional Fees',
       'Sub_Budget_Total General Expenditure',
       'Sub_Budget_Total Indirect Costs',
       'Sub_Budget_Total Land, vehicles & equipment',
       'Sub_Budget_Total Logistics, Transport & Storage',
       'Sub_Budget_Total Operational Provisions',
       'Sub_Budget_Total Programme Support', 'Sub_Budget_Total Personnel',
       'Sub_Budget_Total Relief items, Construction, Supplies',
       'Sub_Budget_Total Services', 'Sub_Budget_Total Supplies',
       'Sub_Budget_Total Transport & Storage',
       'Sub_Budget_Total Workshops & Training', 'TOTAL EXPENDITURE',
       'Sub_Exp_Total General Expenditure',
       'Sub_Exp_Total Land, vehicles & equipment',
       'Sub_Exp_Total Logistics, Transport & Storage',
       'Sub_Exp_Total Operational Provisions',
       'Sub_Exp__Total Programme Support', 'Sub_Exp__Total Services',
       'Sub_Exp_Total Supplies', 'Sub_Exp_Total Transport & Storage',
       'Sub_Exp_Total Workshops & Training', 'Sub_Exp_Total Indirect Costs',
       'Sub_Exp_Total Personnel',
       'Sub_Exp_Total Relief items, Construction, Supplies',
       'Sub_Exp_Consultants Professional Fees',
       'Var_Grand Total', 'Var_Grand Total Distributed', 'Budget_Grand Total',
       'Expenditure']


# Sample function to process inputs and generate data
def process_data(amount_funded, num_beneficiaries, disaster_type, country, region,df_merge_nonzero,cols):
    # You can replace this with your data processing logic
    df_country,df_region,img_data_country,img_data_region=get_simlilar_data(amount_funded, num_beneficiaries, disaster_type, country, region,df_merge_nonzero,cols)
    
    return df_country.reset_index(drop=True),df_region.reset_index(drop=True),img_data_country,img_data_region

@app.route('/')
def index():
    return render_template('home.html')

@app.route('/team')
def team():
    return render_template('team.html')

@app.route('/get_subcategories', methods=['POST'])
def get_subcategories():
    selected_category = request.form.get('selected_category')
    subcategory_list = grouped_regions.get(selected_category, [])
    return jsonify(subcategory_list)

# Route to Similar Disasters Page
@app.route('/dataframe', methods=['GET', 'POST'])
def clustering():
    if request.method == 'POST':
#         print(request.form)
        amount_funded = float(request.form['amount_funded'])
        num_beneficiaries = float(request.form['num_beneficiaries'])
        disaster_type = request.form['disaster_type']
        region = request.form['region']
        country = request.form['subcategory']
        cols_all=['funded_ben','amount_funded','amount_requested','num_beneficiaries','atype_display','country.name',
              'country.society_name','dtype.name','region.region_name','start_year']
        try:
            if(disaster_type=='Flood'):
                cols=cols_all + cols_flood
            else:
                cols = cols_all
            df_country,df_region,img_data_country,img_data_region = process_data(amount_funded, num_beneficiaries, disaster_type, country, region,df_merge_nonzero,cols)
            return render_template('dataframe.html',amount_funded=amount_funded,num_beneficiaries=num_beneficiaries,\
                                   disaster_type=disaster_type,region=region,country=country,\
                                   disaster_types=disaster_types,regions=regions, df_country=df_country.to_html(classes='table table-bordered table-striped table-hover'),img_data_country=img_data_country, df_region=df_region.to_html(classes='table table-bordered table-striped table-hover'),img_data_region=img_data_region)
        except:
            df_country=None
            df_region=None
            return render_template('dataframe.html',disaster_types=disaster_types,regions=regions, df_country=None, img_data_country=None)
    return render_template('dataframe.html',first_load=True,disaster_types=disaster_types,regions=regions,df_region=None, img_data_country=None)

@app.route('/document')
def document():
    # Replace 'document.pdf' with the path to your document file
    document_path = 'Red Cross Hackathon.docx'
    return send_file(document_path, as_attachment=True)
#     return send_from_directory('/', document_path)
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
